In [1]:
import os
print(f'Current directory: {os.getcwd()}')
os.chdir('../')
print(f'Changed to directory: {os.getcwd()}')
print(f'Config file exists: {os.path.exists("config/config.yaml")}')

Current directory: c:\Users\tumom\OneDrive\Desktop\END TO END -NLP\research
Changed to directory: c:\Users\tumom\OneDrive\Desktop\END TO END -NLP
Config file exists: True


In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int
    per_device_eval_batch_size: int
    max_source_length: int
    max_target_length: int
    val_max_target_length: int
    num_beams: int
    load_best_model_at_end: bool
    greater_is_better: bool
    metric_for_best_model: str
    save_total_limit: int
    report_to: str
    push_to_hub: bool
    hub_model_id: str
    hub_token: str
    hub_strategy: str
    hub_private_repo: bool
    hub_always_push: bool
    hub_revision: str


In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [4]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=config.num_train_epochs,
            warmup_steps=config.warmup_steps,
            per_device_train_batch_size=config.per_device_train_batch_size,
            weight_decay=config.weight_decay,
            logging_steps=config.logging_steps,
            evaluation_strategy=config.evaluation_strategy,
            eval_steps=config.eval_steps,
            save_steps=config.save_steps,
            gradient_accumulation_steps=config.gradient_accumulation_steps,
            per_device_eval_batch_size=config.per_device_eval_batch_size,
            max_source_length=config.max_source_length,
            max_target_length=config.max_target_length,
            val_max_target_length=config.val_max_target_length,
            num_beams=config.num_beams,
            load_best_model_at_end=config.load_best_model_at_end,
            greater_is_better=config.greater_is_better,
            metric_for_best_model=config.metric_for_best_model,
            save_total_limit=config.save_total_limit,
            report_to=config.report_to,
            push_to_hub=config.push_to_hub,
            hub_model_id=config.hub_model_id,
            hub_token=config.hub_token,
            hub_strategy=config.hub_strategy,
            hub_private_repo=config.hub_private_repo,
            hub_always_push=config.hub_always_push,
            hub_revision=config.hub_revision
        )

        return model_trainer_config

In [5]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk, load_dataset
import torch


c:\Users\tumom\OneDrive\Desktop\END TO END -NLP\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, # output directory
            num_train_epochs=self.config.num_train_epochs, # total number of training epochs
            warmup_steps=self.config.warmup_steps, # number of warmup steps for learning rate scheduler
            per_device_train_batch_size=self.config.per_device_train_batch_size, # batch size for training
            weight_decay=self.config.weight_decay, # strength of weight decay
            logging_steps=self.config.logging_steps, # number of steps for logging
            eval_strategy=self.config.evaluation_strategy, # evaluation strategy to adopt during training
            eval_steps=self.config.eval_steps, # number of steps for evaluation
            save_steps=self.config.save_steps, # number of steps for saving the model
            gradient_accumulation_steps=self.config.gradient_accumulation_steps, # number of steps for gradient accumulation
            per_device_eval_batch_size=self.config.per_device_eval_batch_size, # batch size for evaluation
            load_best_model_at_end=self.config.load_best_model_at_end, # whether to load the best model at the end of training
            greater_is_better=self.config.greater_is_better, # whether a greater metric value is better
            metric_for_best_model=self.config.metric_for_best_model, # metric to use for best model selection
            save_total_limit=self.config.save_total_limit, # maximum number of checkpoints to save
            report_to=self.config.report_to, # reporting destination
            push_to_hub=self.config.push_to_hub, # whether to push the model to the hub
            hub_model_id=self.config.hub_model_id if self.config.push_to_hub else None # model id for the hub
        )
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        trainer.train()
        trainer.evaluate(dataset_samsum_pt["validation"])
        trainer.save_model(self.config.root_dir)
        trainer.push_to_hub()
        tokenizer.save_pretrained(self.config.root_dir)
        model_pegasus.save_pretrained(self.config.root_dir)


In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\tumom\AppData\Local\Temp\ipykernel_9208\599082730.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,
c:\Users\tumom\OneDrive\Desktop\END TO END -NLP\venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss
